In [1]:
import keras

Using TensorFlow backend.


In [2]:
print('keras: ', keras.__version__)

keras:  2.0.8


In [3]:
import keras
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.preprocessing import image
from keras.engine import Layer
from keras.applications.inception_resnet_v2 import preprocess_input
from keras.layers import Conv2D, UpSampling2D, InputLayer, Conv2DTranspose, Input, Reshape, merge, concatenate
from keras.layers import Activation, Dense, Dropout, Flatten
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential, Model
from keras.layers.core import RepeatVector, Permute
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray, xyz2lab
from skimage.io import imsave
import numpy as np
import os
import random
import tensorflow as tf

In [4]:
# Get images
image_raw = img_to_array(load_img('dog.jpg'))
image_raw = np.array(image_raw, dtype=float)
#Load weights
inception = InceptionResNetV2(weights='imagenet', include_top=True)

In [5]:
X = rgb2lab(1.0/255*image_raw)[:,:,0]
Y = rgb2lab(1.0/255*image_raw)[:,:,1:]
Y /= 128
X = X.reshape(1, 256, 256, 1)
Y = Y.reshape(1, 256, 256, 2)

In [6]:
def conv_stack(data, filters, s):
        output = Conv2D(filters, (3, 3), strides=s, activation='relu', padding='same')(data)
        output = BatchNormalization()(output)
        return output

#Create inception embedding
img_path = 'dog.jpg'
img = image.load_img(img_path, target_size=(299, 299))
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0)
img = preprocess_input(img)
embed = inception.predict(img)
embed_input = Input(shape=(1000,))

#Encoder
encoder_input = Input(shape=(256, 256, 1,))
encoder_output = conv_stack(encoder_input, 64, 2)
encoder_output = conv_stack(encoder_output, 128, 2)
encoder_output = conv_stack(encoder_output, 256, 2)
encoder_output = conv_stack(encoder_output, 512, 1)
encoder_output = conv_stack(encoder_output, 256, 1)

#Fusion
# y_mid: (None, 256, 28, 28)
fusion_output = RepeatVector(32 * 32)(embed_input) 
fusion_output = Permute((2, 1))(fusion_output) 
fusion_output = Reshape(([32, 32, 1000]))(fusion_output)
fusion_output = concatenate([fusion_output, encoder_output], axis=3) 
fusion_output = Conv2D(256, (1, 1), activation='relu')(fusion_output) 

#Decoder
decoder_output = UpSampling2D((2, 2))(fusion_output)
decoder_output = conv_stack(decoder_output, 128, 1)
decoder_output = conv_stack(decoder_output, 64, 1)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = conv_stack(decoder_output, 32, 1)
decoder_output = conv_stack(decoder_output, 16, 1)
decoder_output = Conv2D(2, (2, 2), activation='tanh', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)

model = Model(inputs=[encoder_input, embed_input], outputs=decoder_output)

In [8]:
# Finish model
model.compile(optimizer='rmsprop', loss='mse')
model.fit([X, embed], Y,
    batch_size=1,
    epochs=1000)

Epoch 1/1000
1/1 [==============================] - 4s - loss: 0.4489
Epoch 2/1000
1/1 [==============================] - 0s - loss: 0.2836
Epoch 3/1000
1/1 [==============================] - 0s - loss: 0.2287
Epoch 4/1000
1/1 [==============================] - 0s - loss: 0.1272
Epoch 5/1000
1/1 [==============================] - 0s - loss: 0.1087
Epoch 6/1000
1/1 [==============================] - 0s - loss: 0.0749
Epoch 7/1000
1/1 [==============================] - 0s - loss: 0.0439
Epoch 8/1000
1/1 [==============================] - 0s - loss: 0.0335
Epoch 9/1000
1/1 [==============================] - 0s - loss: 0.0338
Epoch 10/1000
1/1 [==============================] - 0s - loss: 0.0284
Epoch 11/1000
1/1 [==============================] - 0s - loss: 0.0267
Epoch 12/1000
1/1 [==============================] - 0s - loss: 0.0256
Epoch 13/1000
1/1 [==============================] - 0s - loss: 0.0272
Epoch 14/1000
1/1 [==============================] - 0s - loss: 0.0248
Epoch 15/1000
1

1/1 [==============================] - 0s - loss: 0.0016
Epoch 230/1000
1/1 [==============================] - 0s - loss: 0.0012
Epoch 231/1000
1/1 [==============================] - 0s - loss: 0.0018
Epoch 232/1000
1/1 [==============================] - 0s - loss: 0.0018
Epoch 233/1000
1/1 [==============================] - 0s - loss: 0.0023
Epoch 234/1000
1/1 [==============================] - 0s - loss: 0.0016
Epoch 235/1000
1/1 [==============================] - 0s - loss: 0.0017
Epoch 236/1000
1/1 [==============================] - 0s - loss: 0.0015
Epoch 237/1000
1/1 [==============================] - 0s - loss: 0.0016
Epoch 238/1000
1/1 [==============================] - 0s - loss: 0.0016
Epoch 239/1000
1/1 [==============================] - 0s - loss: 0.0013
Epoch 240/1000
1/1 [==============================] - 0s - loss: 0.0016
Epoch 241/1000
1/1 [==============================] - 0s - loss: 0.0023
Epoch 242/1000
1/1 [==============================] - 0s - loss: 0.0020
Epoch 2

1/1 [==============================] - 0s - loss: 7.3093e-04
Epoch 453/1000
1/1 [==============================] - 0s - loss: 0.0014
Epoch 454/1000
1/1 [==============================] - 0s - loss: 0.0011
Epoch 455/1000
1/1 [==============================] - 0s - loss: 0.0011
Epoch 456/1000
1/1 [==============================] - 0s - loss: 0.0012
Epoch 457/1000
1/1 [==============================] - 0s - loss: 9.1827e-04
Epoch 458/1000
1/1 [==============================] - 0s - loss: 6.7318e-04
Epoch 459/1000
1/1 [==============================] - 0s - loss: 6.5021e-04
Epoch 460/1000
1/1 [==============================] - 0s - loss: 0.0011
Epoch 461/1000
1/1 [==============================] - 0s - loss: 6.6726e-04
Epoch 462/1000
1/1 [==============================] - 0s - loss: 0.0015
Epoch 463/1000
1/1 [==============================] - 0s - loss: 0.0017
Epoch 464/1000
1/1 [==============================] - 0s - loss: 0.0012
Epoch 465/1000
1/1 [==============================] - 0s - 

1/1 [==============================] - 0s - loss: 6.9790e-04
Epoch 672/1000
1/1 [==============================] - 0s - loss: 0.0011
Epoch 673/1000
1/1 [==============================] - 0s - loss: 8.5519e-04
Epoch 674/1000
1/1 [==============================] - 0s - loss: 6.9614e-04
Epoch 675/1000
1/1 [==============================] - 0s - loss: 7.2848e-04
Epoch 676/1000
1/1 [==============================] - 0s - loss: 9.4109e-04
Epoch 677/1000
1/1 [==============================] - 0s - loss: 6.1742e-04
Epoch 678/1000
1/1 [==============================] - 0s - loss: 7.9004e-04
Epoch 679/1000
1/1 [==============================] - 0s - loss: 6.5132e-04
Epoch 680/1000
1/1 [==============================] - 0s - loss: 6.8588e-04
Epoch 681/1000
1/1 [==============================] - 0s - loss: 9.9917e-04
Epoch 682/1000
1/1 [==============================] - 0s - loss: 8.1271e-04
Epoch 683/1000
1/1 [==============================] - 0s - loss: 8.3827e-04
Epoch 684/1000
1/1 [===========

1/1 [==============================] - 0s - loss: 6.3976e-04
Epoch 888/1000
1/1 [==============================] - 0s - loss: 7.4644e-04
Epoch 889/1000
1/1 [==============================] - 0s - loss: 5.7753e-04
Epoch 890/1000
1/1 [==============================] - 0s - loss: 4.7277e-04
Epoch 891/1000
1/1 [==============================] - 0s - loss: 6.1158e-04
Epoch 892/1000
1/1 [==============================] - 0s - loss: 9.0013e-04
Epoch 893/1000
1/1 [==============================] - 0s - loss: 9.0569e-04
Epoch 894/1000
1/1 [==============================] - 0s - loss: 9.1043e-04
Epoch 895/1000
1/1 [==============================] - 0s - loss: 8.5959e-04
Epoch 896/1000
1/1 [==============================] - 0s - loss: 6.6838e-04
Epoch 897/1000
1/1 [==============================] - 0s - loss: 6.0348e-04
Epoch 898/1000
1/1 [==============================] - 0s - loss: 8.0146e-04
Epoch 899/1000
1/1 [==============================] - 0s - loss: 6.1663e-04
Epoch 900/1000
1/1 [=======

1/1 [==============================] - 0s - loss: 6.8964e-04
Epoch 996/1000
1/1 [==============================] - 0s - loss: 8.3884e-04
Epoch 997/1000
1/1 [==============================] - 0s - loss: 6.3481e-04
Epoch 998/1000
1/1 [==============================] - 0s - loss: 4.4160e-04
Epoch 999/1000
1/1 [==============================] - 0s - loss: 0.0010
Epoch 1000/1000
1/1 [==============================] - 0s - loss: 9.3028e-04


In [9]:
output = model.predict([X, embed])
output *= 128
# Output colorizations
cur = np.zeros((256, 256, 3))
cur[:,:,0] = X[0][:,:,0]
cur[:,:,1:] = output[0]
imsave("img_result.png", lab2rgb(cur))
imsave("img_gray_version.png", rgb2gray(lab2rgb(cur)))

/Users/ewallner/.brew/lib/python3.6/site-packages/skimage/color/colorconv.py:985: UserWarning: Color data out of range: Z < 0 in 16 pixels
  warn('Color data out of range: Z < 0 in %s pixels' % invalid[0].size)
/Users/ewallner/.brew/lib/python3.6/site-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
/Users/ewallner/.brew/lib/python3.6/site-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint16
  .format(dtypeobj_in, dtypeobj_out))


In [ ]:
#embed_output = UpSampling2D((2, 2))(embed_input)
#embed_output = UpSampling2D((2, 2))(embed_input)
# embed_output= Conv2D(512, (3, 3), activation="relu", strides=2)(embed_input)
# embed_output= Conv2D(512, (3, 3), activation="relu", strides=2)(embed_output)
# embed_output = Dense(1024, activation='relu')(embed_output)
# embed_output = Dense(512, activation='relu')(embed_output)
# embed_output = Dense(256, activation='relu')(embed_output)
# embed_output = Reshape(([256,]))(embed_output)